In [2]:
import edgedb
import gradio as gr

conn = edgedb.create_client()

In [3]:
jsonResponse = conn.query("""
    SELECT Treatment {
        name,
        used_for: {
            name
        },
        requires_devices: {
            name
        }
    }
""")
print(jsonResponse)


[Object{name := 'Rest, Avoid painful movements ', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'Physical therapy exercises', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'Nonsteroidal anti-inflammatory drugs (NSAIDs)', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'Corticosteroid injections in the shoulder', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'Moist heat', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'Icing 20 to 30 minutes 2-3 x day', used_for := [], requires_devices := [Object{name := 'Shoulder Ice Wrap '}, Object{name := 'Shoulder Brace'}]}, Object{name := 'PT - External rotat

In [5]:
import re
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import edgedb
import gradio as gr

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Connect to EdgeDB

# Function to find the most similar symptom in the database
def find_similar_symptom(user_input):
    # Split user input into individual symptoms
    user_symptoms = re.split(r'\s*,\s*', user_input)

    # Retrieve all symptoms from the database
    query = """
        SELECT default::Symptom {
            description
        };
    """
    symptoms = conn.query(query)

    similar_symptoms = []
    # Calculate similarity score for each user symptom
    for user_symptom in user_symptoms:
        # Preprocess user input
        user_input_tokens = nlp(user_symptom.lower())

        similarity_scores = {}
        # Calculate similarity with each symptom description
        for symptom in symptoms:
            symptom_tokens = nlp(symptom.description.lower())
            similarity = cosine_similarity(np.array([user_input_tokens.vector]), np.array([symptom_tokens.vector]))[0][0]
            similarity_scores[symptom.description] = similarity

        # Find the most similar symptom
        most_similar_symptom = max(similarity_scores, key=similarity_scores.get)
        similar_symptoms.append(most_similar_symptom)

    return similar_symptoms

# Function to find the most similar injury in the database
def find_similar_injury(similar_symptoms):
    # Retrieve all injuries from the database
    query = """
        SELECT default::Injury {
            name,
            symptoms: {
                description
            },
            treatments: {
                name
            }
        };
    """
    injuries = conn.query(query)

    similarity_scores = {}
    # Calculate similarity score for each injury
    for injury in injuries:
        injury_symptoms = [symptom.description.lower() for symptom in injury.symptoms]
        similarity = cosine_similarity([nlp(" ".join(similar_symptoms)).vector], [nlp(" ".join(injury_symptoms)).vector])[0][0]
        similarity_scores[injury.name] = {
            "similarity": similarity,
            "treatments": [treatment.name for treatment in injury.treatments]
        }

    # Find the most similar injury
    most_similar_injury_name = max(similarity_scores, key=lambda x: similarity_scores[x]["similarity"])
    most_similar_injury = similarity_scores[most_similar_injury_name]
    return most_similar_injury_name, most_similar_injury["treatments"]

# Create Gradio interface
symptoms_textbox = gr.Textbox(label="Enter Symptoms", lines=3)

def get_similar_injury(user_input):
    # Find similar symptoms
    similar_symptoms = find_similar_symptom(user_input)
    
    # Find similar injury and treatments based on similar symptoms
    similar_injury, treatments = find_similar_injury(similar_symptoms)
    
    return similar_injury, treatments

iface = gr.Interface(get_similar_injury, inputs=symptoms_textbox, outputs=["text", "text"], title="Injury Diagnosis", description="Enter your symptoms to get a diagnosis.")
iface.launch()


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.20.1, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


In [6]:
user_input = "swelling, grinding sound, limited motion"  # Example user input
similar_symptoms = find_similar_symptom(user_input)
similar_injury = find_similar_injury(similar_symptoms)
print("User Input:", user_input)
print("Most Similar Symptoms in Database:", similar_symptoms)
print("Most Similar Injury:", similar_injury)


User Input: swelling, grinding sound, limited motion
Most Similar Symptoms in Database: ['swelling', 'hear/feel a crunching or grinding sound (crepitus) when you try to move your arm.', 'Extremely limited range of motion  ']
Most Similar Injury: ('Sternoclavicular (SC) Joint Disorders', ['Nonsteroidal anti-inflammatory drugs (NSAIDs)', 'Immobilization', 'Activity Modification Avoid Painful movements '])
